In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
# get tf version
print(tf.__version__)
from keras.layers import Dense, LSTM, Input
from keras import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

2.16.1


In [25]:
# fix random seed for reproducibility
tf.random.set_seed(7)

In [26]:
df_avg = pd.read_csv('data/new_avg.csv')
date_time = pd.to_datetime(df_avg.pop('date'), format='%Y-%m-%d')
ds_avg = df_avg.to_numpy()
ds_avg = ds_avg.astype("float32")

In [27]:
# scale data

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
ds_avg = scaler.fit_transform(ds_avg)

In [28]:
# split into train and test sets
train_size = int(len(ds_avg) * 0.67)
test_size = len(ds_avg) - train_size
train, test = ds_avg[0:train_size,:], ds_avg[train_size:len(ds_avg),:]
print(len(train), len(test))
train.shape

64 32


(64, 19)

In [29]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=6):
	data_x, data_y = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back)]
		data_x.append(a)
		data_y.append(dataset[i + look_back])
	return np.array(data_x), np.array(data_y)

In [30]:
lookback = 6
train_X, train_y = create_dataset(train, lookback)
test_X, test_y = create_dataset(test, lookback)
train_X.shape

(57, 6, 19)

In [31]:
# reshape into [samples, time steps, features]

# train_X = np.reshape(train_X, (train.shape[0], 6, train.shape[1]))
# test_X = np.reshape(test_X, (test.shape[0], 6, tes.shape[1]))
train_X.shape

(57, 6, 19)

In [45]:
# create and fit the LSTM network
model = Sequential()
# model.add(Input(shape=train_X.shape) )
model.add(LSTM(4))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam') # look at other loss functions
model.fit(train_X, train_y, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
57/57 - 5s - 93ms/step - loss: 0.0742
Epoch 2/100
57/57 - 0s - 5ms/step - loss: 0.0517
Epoch 3/100
57/57 - 0s - 4ms/step - loss: 0.0512
Epoch 4/100
57/57 - 0s - 5ms/step - loss: 0.0508
Epoch 5/100
57/57 - 0s - 6ms/step - loss: 0.0505
Epoch 6/100
57/57 - 0s - 4ms/step - loss: 0.0502
Epoch 7/100
57/57 - 0s - 4ms/step - loss: 0.0500
Epoch 8/100
57/57 - 0s - 6ms/step - loss: 0.0499
Epoch 9/100
57/57 - 0s - 5ms/step - loss: 0.0497
Epoch 10/100
57/57 - 0s - 5ms/step - loss: 0.0496
Epoch 11/100
57/57 - 0s - 6ms/step - loss: 0.0495
Epoch 12/100
57/57 - 0s - 6ms/step - loss: 0.0494
Epoch 13/100
57/57 - 0s - 6ms/step - loss: 0.0494
Epoch 14/100
57/57 - 0s - 6ms/step - loss: 0.0493
Epoch 15/100
57/57 - 0s - 6ms/step - loss: 0.0492
Epoch 16/100
57/57 - 0s - 5ms/step - loss: 0.0492
Epoch 17/100
57/57 - 0s - 5ms/step - loss: 0.0491
Epoch 18/100
57/57 - 0s - 4ms/step - loss: 0.0491
Epoch 19/100
57/57 - 0s - 4ms/step - loss: 0.0490
Epoch 20/100
57/57 - 0s - 4ms/step - loss: 0.0490
Epoch 21

In [47]:
# make predictions
trainPredict = model.predict(train_X)
testPredict = model.predict(test_X)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([train_y])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([test_y])
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(train_y[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(test_y[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

test_X.shape

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


ValueError: non-broadcastable output operand with shape (57,1) doesn't match the broadcast shape (57,19)